In [1]:
import os
import pandas as pd
import numpy as np
import sys
import csv
from time import strptime
import json
from datetime import datetime as dt
from datetime import date
from statistics import mean
import math

In [3]:
df = pd.read_csv('14 ratings.csv')

In [4]:
metadata = pd.read_csv('MovieSummaries/movie.metadata.tsv', sep='\t', header=None)

In [5]:
metadata.columns = ['id_wiki','id_freebase', 'name', 'release', 'revenue', 'runtime', 'langs', 'countries', 'genres']

In [6]:
metadata.shape

(81741, 9)

In [7]:
file1 = open("MovieSummaries/plot_summaries.txt","r+")

In [8]:
plots = file1.read().split('\n')

In [9]:
plot_ids = [s.split('\t')[0] for s in plots]

In [10]:
df_plots = pd.DataFrame(data = {'id_wiki': plot_ids, 'plot': plots}).drop(42306, axis=0)

In [11]:
df_plots['plot'] = [plot.split('\t', 1)[1] for plot in list(df_plots['plot'])]

In [12]:
df_plots.id_wiki = df_plots.id_wiki.astype(np.int64)

In [13]:
df_plots.shape

(42306, 2)

In [14]:
merged = df_plots.merge(metadata, how='inner', on=['id_wiki'])

In [15]:
merged['year'] = [release.split('-', 1)[0] if not pd.isna(release) else np.nan for release in merged.release]

In [16]:
merged['year'] = [int(y) if not pd.isna(y) else np.nan for y in merged.year]

In [17]:
merged.head()

,id_wiki,plot,id_freebase,name,release,revenue,runtime,langs,countries,genres,year
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha...",/m/076w2lb,Taxi Blues,1990-09-07,NaN,110.0,"{""/m/06b_j"": ""Russian Language""}","{""/m/0f8l9c"": ""France"", ""/m/05vz3zq"": ""Soviet ...","{""/m/07s9rl0"": ""Drama"", ""/m/03q4nz"": ""World ci...",1990.0
1,31186339,The nation of Panem consists of a wealthy Capi...,/m/0gkz15s,The Hunger Games,2012-03-12,686533290.0,142.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/03btsm8"": ""Action/Adventure"", ""/m/06n90"":...",2012.0
2,20663735,Poovalli Induchoodan is sentenced for six yea...,/m/051zjwb,Narasimham,2000,NaN,175.0,"{""/m/0999q"": ""Malayalam Language""}","{""/m/03rk0"": ""India""}","{""/m/04t36"": ""Musical"", ""/m/02kdv5l"": ""Action""...",2000.0
3,2231378,"The Lemon Drop Kid , a New York City swindler,...",/m/06xtz3,The Lemon Drop Kid,1951-03-08,2300000.0,91.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06qm3"": ""Screwball comedy"", ""/m/01z4y"": ""...",1951.0
4,595909,Seventh-day Adventist Church pastor Michael Ch...,/m/02tqm5,A Cry in the Dark,1988-11-03,6908797.0,121.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",1988.0


In [18]:
df.columns

Index(['budget', 'META__id', 'PROCESS__original_language',
       'PROCESS__original_title', 'PROCESS__overview', 'revenue', 'runtime',
       'spoken_languages', 'PROCESS__title', 'META__year',
       ...
       'production_company_3_avg_profitability',
       'production_company_3_avg_revenue', 'collection_name_avg_profit',
       'collection_name_avg_revenue', 'competition', 'rating__g', 'rating__pg',
       'rating__pg-13', 'rating__r', 'rating__nc-17'],
      dtype='object', length=305)

In [19]:
df.shape

(7494, 305)

In [20]:
merged.columns = ['id_wiki', 'plot', 'id_freebase', 'PROCESS__title', 'release', 'revenue', 'runtime', 'langs', 'countries', 'genres', 'META__year']

In [21]:
df.META__year = df.META__year.astype(np.float64)

In [22]:
merged_df = df.merge(merged, how='inner', on=['PROCESS__title', 'META__year'])

In [23]:
merged_df.shape

(4520, 314)

In [24]:
c_longer = 0
for i, row in merged_df.iterrows():
    if len(row['plot']) > len(row['PROCESS__overview']):
        c_longer+=1
c_longer

4441

In [25]:
4441/4520

0.9825221238938053

In [26]:
for i, row in merged_df.iterrows():
    if len(row['plot']) < len(row['PROCESS__overview']):
        merged_df.loc[merged_df.META__id == row.META__id, 'plot'] = row['PROCESS__overview']

In [27]:
merged_df.shape

(4520, 314)

In [28]:
merged_df.drop_duplicates('META__id').shape

(4503, 314)

In [29]:
plot_df = merged_df[['META__id', 'plot']]

In [30]:
plot_df.head()

,META__id,plot
0,5,"The film is set on New Year's Eve, and starts ..."
1,6,Frank Wyatt leaves his wife and baby daughter...
2,12,"Two clownfish, Marlin and his wife Coral are a..."
3,13,"As he waits at a bus stop, Forrest Gump star..."
4,14,Lester Burnham is a middle-aged magazine writ...


In [31]:
plot_df['plot'] = plot_df['plot'].str.replace('$', '')

/home/stas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
plot_df.to_csv('plot_df.csv', mode='w+', index=False)